In [ ]:
import csv
import time
from typing import List
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# WebDriverの設定
driver_path = "\\Users\\katus\\テスト\\chromedriver.exe"
chrome_options = Options()
chrome_options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
service = Service(driver_path)
browser = webdriver.Chrome(service=service, options=chrome_options)

# CSVファイルから企業ページURLを読み込む
with tqdm(total=3) as pbar:
    with open('リクナビNEXT企業ページURL.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # ヘッダー行をスキップ

        for _ in range(3):
            row = next(reader)
            if not row:
                break

            company_id = row[0]
            url = f'https://next.rikunabi.com{company_id}'
            job_count = []

            # 企業ページを開く
            browser.get(url)
            time.sleep(3)  # ページ読み込み待ち

            # 情報取得用のリストを初期化
            company_value = []
            industry_value = []
            address_value = []

            # ページのHTMLを取得し、BeautifulSoupで解析
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')

            try:
                # ログインボタンをクリックして企業情報を取得
                elem_login_btn = browser.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[1]/div/div[2]/ul/li[1]/div[3]/div/div[2]/div/div[1]/a').click()
            except NoSuchElementException:
                elem_login_btn = browser.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[2]/div/div[2]/ul/li/div[1]/h2/a').click()
                time.sleep(3)
                browser.switch_to.window(browser.window_handles[1])

            # 企業名を取得
            company_name_element = browser.find_element(By.CSS_SELECTOR, "p.rnn-offerCompanyName")
            company_name = company_name_element.text.strip()
            company_value.append(company_name)

            # 業種を取得
            industry_element = browser.find_element(By.XPATH, "//th[text()='業種']/following-sibling::td/p")
            industry = industry_element.text.strip().split('/')[0]
            industry_value.append(industry)

            # 連絡先を取得
            contact_element = browser.find_element(By.XPATH, "//th[text()='連絡先']/following-sibling::td/p")
            contact = contact_element.text.strip()
            address_value.append(contact)

            # データを辞書に格納し、リストに追加
            data = {
                "company_name": company_value,
                "industry": industry_value,
                "address": address_value,
                "jobcount": job_count
            }
            all_data.append(data)

            # ブラウザを閉じてメインウィンドウに戻る
            browser.close()
            browser.switch_to.window(browser.window_handles[0])
            pbar.update(1)

# 取得したデータをDataFrameに変換してCSV出力
df = pd.DataFrame(all_data)
df['company_name'] = df['company_name'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else "")
df['industry'] = df['industry'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else "")
df['address'] = df['address'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else "")
df['jobcount'] = df['jobcount'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else "")
df.to_csv('20240326.csv', index=False)
